In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 19 20:21:08 2018

@author: Vicky
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from PIL import Image

import pandas as pd
import numpy as np
import os, time, glob

import tensorflow as tf
from sklearn.model_selection import train_test_split

from os import listdir
from os.path import isfile, join
import numpy as np
import cv2, os
import itertools
import random
import time
from sklearn.model_selection import train_test_split

DATA_SIZE = 100000

print("Imported Requirements")
## Import images
mypath='V:\\Sem_2\\674\\pa1\\Project_1_SML\\Dataset[Without-Features]\\AND_Images[WithoutFeatures]'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
    images[n] = cv2.imread( join(mypath,onlyfiles[n]),cv2.IMREAD_GRAYSCALE )
print("Imported Images")

## Crop images ##
images = [i[~np.all(i == 255, axis=1)][:,~np.all(i[~np.all(i == 255, axis = 1)] == 255, axis=0)] for i in images]
print("Cropped Images")

## Resize images ##
shapes = [i.shape for i in images]
height = [i[0] for i in shapes]
length = [i[1] for i in shapes]
avg_height = int(sum(height)/len(height))
avg_length = int(sum(length)/len(length))
print(avg_length, avg_height)
images = [cv2.resize(images[i],(avg_length, avg_height)) for i in range(len(images))]
print("Resized Images")

## Write Processed images ##
processed_directory = "Processed/"
subtracted_directory = "Subtracted/"
if not os.path.exists(processed_directory):
    os.makedirs(processed_directory)
if not os.path.exists(subtracted_directory):
    os.makedirs(subtracted_directory)
save = [cv2.imwrite(processed_directory+onlyfiles[i],images[i]) for i in range(len(images))]
print("Processed and Written Images")

## Selecting similar sample ##
authors = sorted(set([i.split("_")[0][:4] for i in onlyfiles]))
author_dict = {}

for i in authors:
    author_dict[i] = []
continue_index = 0
sorted_keys = sorted(author_dict.keys())
for i in sorted_keys:
    for j in onlyfiles[continue_index:]:
        if i in j:
            author_dict[i].append(j)
        else:
            continue_index = onlyfiles.index(j)
            break

permutated_dict = {}
for i in authors:
    permutated_dict[i] = []
for i in author_dict.keys():
    for r in itertools.product(author_dict[i], author_dict[i]):
        if (r[0] != r[1]) and ([r[1],r[0]] not in permutated_dict[i]):
            permutated_dict[i].append([r[0],r[1]])
            
similar = [j for i in permutated_dict.keys() for j in permutated_dict[i]]
similar_sample = random.sample(similar,int(DATA_SIZE/2) if int(DATA_SIZE/2) < len(similar) else len(similar))
print("Similar Sample generated")

## Selecting different sample ##
different_keys = [i.split("*_*") for i in set(["*_*".join(sorted([i,j])) for i,j in itertools.product(author_dict.keys(),author_dict.keys()) if i!=j])]
diff_keys_sample = random.sample(different_keys,int(DATA_SIZE/2) if int(DATA_SIZE/2) < len(different_keys) else len(different_keys))
different_sample = [[random.choice(author_dict[i[0]]),random.choice(author_dict[i[1]])] for i in diff_keys_sample]
print("Different Sample generated")

Imported Requirements
Imported Images
Cropped Images
150 65
Resized Images
Processed and Written Images
Similar Sample generated
Different Sample generated


In [3]:
similar_sample_data = [(images[onlyfiles.index(i[0])],images[onlyfiles.index(i[1])]) for i in similar_sample]
print ("Similar sample done")


Similar sample done


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [11]:
print(similar_sample_data[105][0])

[[255 255 255 ..., 255 255 255]
 [255 255 255 ...,  62  68 255]
 [255 255 255 ...,  25  32 255]
 ..., 
 [255 255 255 ...,  25  32 255]
 [255 255 255 ...,  60  32 255]
 [255 255 255 ..., 243  32 255]]


In [12]:
different_sample_data = [(images[onlyfiles.index(i[0])],images[onlyfiles.index(i[1])]) for i in different_sample]
print ("different sample data generated")

different sample data generated


In [28]:
## Save Not_subtracted data ##
not_subtracted_directory = "Not_Subtracted/"
if not os.path.exists(not_subtracted_directory):
    os.makedirs(not_subtracted_directory)
save = [cv2.imwrite(not_subtracted_directory+str(i)+ "$0___" +similar_sample[i][0].split(".")[0]+"__"+similar_sample[i][1],similar_sample_data[i][0]) for i in range(len(similar_sample_data))]
save = [cv2.imwrite(not_subtracted_directory+str(i)+ "$1___" +similar_sample[i][0].split(".")[0]+"__"+similar_sample[i][1],similar_sample_data[i][1]) for i in range(len(similar_sample_data))]
save = [cv2.imwrite(not_subtracted_directory+str(50000+i)+ "$0___" +different_sample[i][0].split(".")[0]+"__"+different_sample[i][1],different_sample_data[i][0]) for i in range(len(different_sample_data))]
save = [cv2.imwrite(not_subtracted_directory+str(50000+i)+ "$1___" +different_sample[i][0].split(".")[0]+"__"+different_sample[i][1],different_sample_data[i][1]) for i in range(len(different_sample_data))]
print("Not Subtracted data saved")

Not Subtracted data saved


In [4]:
## Import images for SIFT keypoints generation
sift_source_path='V:\\Sem_2\\674\\pa1\\Project_1_SML\\Not_Subtracted'
sift_source_files = [ f for f in listdir(sift_source_path) if isfile(join(sift_source_path,f)) ]
sift_source_images = np.empty(len(sift_source_files), dtype=object)
for n in range(0, len(sift_source_files)):
    sift_source_images[n] = cv2.imread( join(sift_source_path,sift_source_files[n]),cv2.IMREAD_GRAYSCALE )
print("Imported sift source Images")

Imported sift source Images


In [6]:
sift_keypoints_directory = "sift_keypoints/"
if not os.path.exists(sift_keypoints_directory):
    os.makedirs(sift_keypoints_directory)

In [9]:
print(len(sift_source_files))

200000


In [11]:
for i in range(len(sift_source_files)):
    sift_img = cv2.imread(join(sift_source_path,sift_source_files[i]))
    sift_gray= cv2.cvtColor(sift_img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    sift_keypoints = sift.detect(sift_gray,None)
    cv2.drawKeypoints(sift_gray,sift_keypoints,sift_img)
    cv2.imwrite(sift_keypoints_directory+'skp_'+sift_source_files[i],sift_img)
    
print("sift keypoint images generated")

sift keypoint images generated


In [68]:
sift_keypoints=[]
sift_descriptors=[]
for i in range(len(sift_source_files)):
    sift_img = cv2.imread(join(sift_source_path,sift_source_files[i]))
    sift_gray= cv2.cvtColor(sift_img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    sift_kp, sift_des = sift.detectAndCompute(sift_img,None)
    sift_keypoints.append(sift_kp)
    sift_descriptors.append(sift_des)

print("Sift keypoints and descriptors generated")
    


Sift keypoints and descriptors generated


In [70]:
print(len(sift_descriptors))

200000


In [67]:
print(sift_descriptors[1].shape[0],sift_descriptors[0].shape)

105 (59, 128)


In [90]:
sift_descriptors[-1]
    


array([[   1.,    0.,    0., ...,    0.,    3.,   83.],
       [   0.,    0.,    0., ...,  128.,    1.,    0.],
       [  63.,   67.,    1., ...,    0.,    0.,    5.],
       ..., 
       [  10.,    1.,    0., ...,    0.,    0.,    0.],
       [   7.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]], dtype=float32)

In [74]:
max([i.shape[0] for i in sift_descriptors])

157

In [77]:
serial=pickle.dumps(sift_descriptors[0],protocol=0)

In [79]:
deserial=pickle.loads(serial)
print(deserial.shape)

(59, 128)


In [99]:
descrptrs=sift_descriptors[:10000]
#print(sift_descriptors[2].shape
with open('descriptors1.pickle', 'wb') as handle:
    pickle.dump(descrptrs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [97]:


# Load data (deserialize)
with open('descriptors.pickle', 'rb') as handle:
    unserialized_data = pickle.load(handle)

print(unserialized_data[0].shape)


(59, 128)


In [98]:
len(unserialized_data)

20

In [105]:
descrptrs=sift_descriptors[160001:200000]
#print(sift_descriptors[2].shape
with open('descriptors7.pickle', 'wb') as handle:
    pickle.dump(descrptrs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [108]:
sift_keypoints[0][0]

<KeyPoint 0000023092EE18D0>

In [109]:
sift_descriptors[0][0]

array([   0.,    0.,    0.,    2.,    8.,    0.,    0.,    0.,    8.,
          0.,    0.,   56.,  152.,    0.,    0.,    2.,  152.,    0.,
          0.,   19.,   86.,    1.,    0.,   74.,   59.,    0.,    0.,
         30.,   23.,    0.,    0.,   23.,    0.,    0.,    0.,    6.,
         26.,    0.,    0.,    0.,   34.,    1.,    1.,   86.,  152.,
          3.,    0.,    2.,  152.,    4.,    1.,   25.,   59.,    1.,
          0.,   34.,   51.,    2.,    1.,  109.,   79.,    0.,    0.,
          8.,    0.,    0.,    0.,    2.,   16.,    0.,    0.,    0.,
         30.,    9.,    1.,   49.,  152.,   11.,    0.,    1.,  152.,
        116.,   36.,   41.,   48.,    3.,    0.,    3.,   41.,   20.,
         34.,  152.,   66.,    0.,    0.,   16.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    2.,    5.,    1.,   21.,
         13.,    0.,    3.,    4.,   17.,   67.,   35.,   19.,    4.,
          6.,   56.,   24.,   12.,   14.,   32.,   33.,    1.,    5.,
         13.,    3.]

In [111]:
sift_descriptors[0]

array([[   0.,    0.,    0., ...,    5.,   13.,    3.],
       [   1.,    4.,   26., ...,    0.,    0.,    2.],
       [  21.,   11.,    0., ...,    0.,    0.,    0.],
       ..., 
       [   0.,    0.,    0., ...,  143.,   20.,    0.],
       [  11.,    8.,    2., ...,    0.,    0.,    2.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]], dtype=float32)

In [156]:
from sklearn.cluster import KMeans

In [157]:
km = KMeans(n_clusters=30, init='k-means++', n_init=1, max_iter=10, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm='auto')

In [158]:
concatenated = []
for i in sift_descriptors[:10000]  + sift_descriptors[100000:110000]:
    concatenated += list(i)

In [159]:
cluster_X = np.array(concatenated)

In [160]:
km.fit(cluster_X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10,
    n_clusters=30, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [185]:
y = km.fit_predict(cluster_X)

In [186]:
cluster_index = list(y)

In [187]:
desc = sift_descriptors[:10000]  + sift_descriptors[100000:110000]

In [188]:
images = []
for i in desc:
    image = [0 for k in range(30)]
    for j in range(i.shape[0]):
        index = cluster_index.pop()
        image[index] += 1
    images.append(image)

In [189]:
len(images)

20000

In [199]:
X_data = []
for i in range(0,20000,2):
    #print(i,i+1)
    X_data.append(abs(np.array(images[i])-np.array(images[i+1])))

In [205]:
X_data = np.array(X_data)

In [206]:
Y_data = np.array([1 for i in range(5000)] + [0 for i in range(5000)])

In [207]:
### Splitting the data into training and testing ###
train_data = {}
test_data = {}
train_data["x"], test_data["x"], train_data["y"], test_data["y"] = train_test_split(X_data, Y_data, test_size=0.2)

In [208]:
train_data

{'x': array([[ 2, 10,  2, ...,  0,  2,  5],
        [ 1, 19,  0, ...,  1,  2,  4],
        [ 3,  0,  2, ...,  1,  1,  3],
        ..., 
        [ 2,  0,  1, ...,  0,  2,  0],
        [ 1,  0,  0, ...,  1,  3,  1],
        [ 3,  3,  1, ...,  3,  2,  3]]), 'y': array([1, 1, 0, ..., 1, 1, 1])}

In [303]:
MAX_ITER = 10000
LEARNING_RATE = 0.0005

def train_logistic_regression(train_X,train_Y,max_iter,learn_rate):
    from sklearn.linear_model import LogisticRegression
    logisticRegr = LogisticRegression(solver = 'sag', multi_class="ovr", max_iter=max_iter, C=learn_rate)
    logisticRegr.fit(train_X, train_Y)
    return logisticRegr

def accuracy_logistic_regression(trained_model, test_X, test_Y):
    predictions = trained_model.predict(test_X)
    score = trained_model.score(test_X, test_Y)
    return score
def sgd_train_logistic_regression(train_X,train_Y,max_iterations,learn_rate):
    from sklearn.linear_model import SGDClassifier
    sgd=SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True,shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, n_iter=2)
    sgd.fit(train_X,train_Y)
    return sgd
    
    

In [304]:
trained_model = train_logistic_regression(train_data["x"],train_data["y"],MAX_ITER,LEARNING_RATE)
print("Accuracy on AND Testing Data: " + str(accuracy_logistic_regression(trained_model,test_data["x"],test_data["y"])))

trained_model_x = sgd_train_logistic_regression(train_data["x"],train_data["y"],MAX_ITER,LEARNING_RATE)
print("Accuracy on AND Testing Data: " + str(accuracy_logistic_regression(trained_model_x,test_data["x"],test_data["y"])))

Accuracy on AND Testing Data: 0.5575
Accuracy on AND Testing Data: 0.4765


In [311]:
#type 1 & Type 2 Error
test_data_type1 = {"x":[],"y":[]}
test_data_type2 = {"x":[],"y":[]}
#calculating Type 1 & Type 2Error
for i in range(2000):
    if(test_data["y"][i]==0):
        test_data_type1["x"].append(test_data["x"][i])
        test_data_type1["y"].append(test_data["y"][i])
    else:
        test_data_type2["x"].append(test_data["x"][i])
        test_data_type2["y"].append(test_data["y"][i])
    
    
    

In [315]:
print("Type1 Error: " + str(1-accuracy_logistic_regression(trained_model,test_data_type1["x"],test_data_type1["y"])))
print("Type2 Error: " + str(1-accuracy_logistic_regression(trained_model,test_data_type2["x"],test_data_type2["y"])))


Type1 Error: 0.391783567134
Type2 Error: 0.493013972056
